# Deepersense: sim data generator

## imports

In [25]:
import os
# import sys
# import rospy
# import rosbag
import rospkg
import glob

In [26]:
from simulation_controller import SimulationController
from scene_generator import SceneGenerator, ColorMap
from launchfile_maker import LaunchFileMaker

## file manager

### cwd

In [27]:
cwd = os.getcwd()

### package path

In [28]:
rospack = rospkg.RosPack()
package_path = rospack.get_path('stonefish_scene_generator')

### iterate files in folder

In [29]:
all_files_path = os.path.join(package_path,'*')
for file_name in glob.glob(all_files_path):
    print(file_name)

/home/ilan/catkin_ws/src/stonefish_scene_generator/package.xml
/home/ilan/catkin_ws/src/stonefish_scene_generator/launch
/home/ilan/catkin_ws/src/stonefish_scene_generator/data
/home/ilan/catkin_ws/src/stonefish_scene_generator/src
/home/ilan/catkin_ws/src/stonefish_scene_generator/include
/home/ilan/catkin_ws/src/stonefish_scene_generator/CMakeLists.txt


### create data folder

In [30]:
data_folder_path = os.path.join(package_path, 'data')
dir_exists = os.path.isdir(data_folder_path)
if not dir_exists:
    os.mkdir(data_folder_path)
    print('data folder created: {}'.format(data_folder_path))

## generate stonefish scene

### generate scene

In [31]:
title = 'basic'
seabed_depth = 15
gen = SceneGenerator(seabed_depth)
gen.add_empty_frame(ColorMap.jet)
gen.add_reefs()
scene = gen.generate()
# print(scene)

### save scene to .scn file

In [32]:
scene_file_path = os.path.join(data_folder_path, title + '.scn')
f = open(scene_file_path, "w")
f.write(scene)
f.close()

## Simulation

### get launch_file_path

In [33]:
launch_file_name = 'auto_generated.launch'
launch_file_path = os.path.join(package_path, 'launch', launch_file_name)
print(launch_file_path)

/home/ilan/catkin_ws/src/stonefish_scene_generator/launch/audo_generated.launch


### edit/make launch file

In [34]:
rate = 300.0
resolution = (1200, 800)
maker = LaunchFileMaker(scene_file_path, rate, resolution)
maker.save_to_file(launch_file_path)

### setup simulation

In [35]:
controller = SimulationController(launch_file_path)

... logging to /home/ilan/.ros/log/36922322-2778-11ec-ab95-a08cfdc4e9f4/roslaunch-ilan-img-ubuntu-32135.log


### launch simulation

In [36]:
controller.run()
controller.wait_for_ros()

started roslaunch server http://ilan-img-ubuntu:37665/

SUMMARY

PARAMETERS
 * /rosdistro: melodic
 * /rosversion: 1.14.10

NODES
  /
    rosbag_record_cam (rosbag/record)
    rviz (rviz/rviz)
    stonefish_simulator (stonefish_ros/parsed_simulator)

auto-starting new master
process[master]: started with pid [4244]
ROS_MASTER_URI=http://localhost:11311
setting /run_id to 36922322-2778-11ec-ab95-a08cfdc4e9f4
process[rosout-13]: started with pid [4255]
started core service [/rosout]
process[stonefish_simulator-14]: started with pid [4262]
process[rviz-15]: started with pid [4263]
process[rosbag_record_cam-16]: started with pid [4269]
[INFO] [1633615781.495687]: started
[rosbag_record_cam-16] killing on exit
[stonefish_simulator-14] killing on exit
[rviz-15] killing on exit


[stonefish_simulator-14] escalating to SIGTERM
[stonefish_simulator-14] escalating to SIGKILL


[rosout-13] killing on exit
[master] killing on exit


Shutdown errors:
 * process[stonefish_simulator-14, pid 4262]: required SIGKILL. May still be running.


## Bag

### open bag file

bag = rosbag.Bag(bagfile, 'r')